## Question Answering su pagine web

Task di QnA con output dei documenti recuperati e usati durante la generazione della risposta finale.

In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer()
loader = WebBaseLoader(
    web_paths=(
        "https://datamasters.it/corsi/",
        "https://datamasters.it/corsi/generative-ai/",
        "https://datamasters.it/corsi/machine-learning-per-tutti/",
        "https://datamasters.it/corsi/data-driven-mindset/",
        "https://datamasters.it/corsi/python-from-zero-to-hero/",
        "https://datamasters.it/corsi/ai-prompt-engineering/",
        "https://datamasters.it/corsi/corsi-data-scientist/",
        "https://datamasters.it/corsi/algotrading-in-python/",
        "https://corsi.datamasters.it/offers/8vpgpbtd/checkout",
        "https://datamasters.it/corsi/matematica-statistica-per-machine-learning/",
        "https://datamasters.it/corsi/machine-learning-starter-kit/",
        "https://datamasters.it/corsi/data-scientist-starter-kit/",
        "https://datamasters.it/corsi/machine-learning-foundations/"
    ),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
docs

[Document(metadata={'source': 'https://datamasters.it/corsi/', 'title': 'Catalogo dei Corsi sull’AI - Data Masters', 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.", 'language': 'it-IT'}, page_content='\n\nCatalogo dei Corsi sull’AI - Data Masters\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nIMPARA A CREARE AGENTI AI\xa0 SCONTO 50% IN SCADENZA\nScopri di piùDataMastersCatalogoCorsiPercorsi di CarrieraNon sai da dove partire? Compila il test di orientamento e ricevi consigli personalizzati Read morePrincipianteIntermedioAvanzatoCategorieCorsi di Generative AICorsi di Data AnalysisCorsi di Data ScienceCorsi di Machine LearningCorsi di Programmazione in PythonTutti i corsiPer le aziendeCommunityCommunity HubBlogChallengesWebinarsAI e

In [3]:
len(docs[0].page_content)

12740

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, chunk_overlap=150, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [5]:
len(all_splits)

1082

In [6]:
len(all_splits[2].page_content)

398

In [7]:
all_splits[42].metadata

{'source': 'https://datamasters.it/corsi/',
 'title': 'Catalogo dei Corsi sull’AI - Data Masters',
 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.",
 'language': 'it-IT',
 'start_index': 9815}

In [8]:
all_splits[42]

Document(metadata={'source': 'https://datamasters.it/corsi/', 'title': 'Catalogo dei Corsi sull’AI - Data Masters', 'description': "Esplora il nostro catalogo e scopri i migliori corsi di formazione continua, sempre aggiornati e con un forte taglio pratico nel campo della Data Science, del Machine Learning e dell'Intelligenza Artificiale.", 'language': 'it-IT', 'start_index': 9815}, page_content='Quanto tempo ho per completare un Percorso di Carriera?Non ci sono scadenze, puoi completarlo secondo i tuoi tempi. Orientativamente indichiamo una durata media di circa 3 mesi con una frequenza di 3-5 ore di studio a settimana.')

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [4]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# metodi di ricerca alternativi:
# `similarity_score_threshold` - come similarity ma impostando una soglia minima di accettazione
# `mmr` = maximal marginal relevance - seleziona un insieme di esempi che sono sia simili all'input che diversi tra loro

In [5]:
retrieved_docs = retriever.invoke("consigliami un corso per iniziare la mia carriera da Data Scientist")

In [6]:
for doc in retrieved_docs:
    print(doc.metadata)

{'description': 'Algoritmi di Machine Learning è il corso per apprendere tutti algoritmi di Machine Learning con Python. Corso Completo Teoria + Esercitazioni', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/machine-learning-foundations/', 'start_index': 117, 'title': 'Algoritmi di Machine Learning - Corso ed Esercitazioni'}
{'description': 'Diventa un Data Scientist con il Data Science Starter Kit, un percorso graduale from 0 to Hero studiato su misura per chi comincia.', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/data-scientist-starter-kit/', 'start_index': 118, 'title': 'Corso online di Data Science e AI per chi parte da zero'}
{'description': 'Impara a Ottimizzare ed Automatizzare le strategie di Trading con Python. Scarica il Sistema di AlgoTrading e aumenta le tue competenze!', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/algotrading-in-python/', 'start_index': 116, 'title': 'Corso di Trading Algoritmico in Python | Data Masters'}
{

In [27]:
print(len(retrieved_docs))

10


In [28]:
retrieved_docs = retriever.invoke("vorrei diventare un Machine Learning Engineer")
for doc in retrieved_docs:
    print(doc.metadata)

{'source': 'https://datamasters.it/corsi/corsi-data-scientist/', 'language': 'it-IT', 'start_index': 379, 'title': 'ML Engineer con la Machine Learning Masterclass | Data Masters', 'description': 'Iscriviti alla Machine Learning Masterclass, il Percorso di Carriera in italiano più avanzato ed aggiornato per diventare un Machine Learning Engineer.'}
{'start_index': 26236, 'title': 'Corso di Machine Learning partendo dalle basi | Data Masters', 'description': 'Il percorso più rapido per imparare il Machine Learning, dalle basi di matematica e statistica, alla programmazione in Python fino agli algoritmi più avanzati.', 'source': 'https://datamasters.it/corsi/machine-learning-starter-kit/', 'language': 'it-IT'}
{'language': 'it-IT', 'description': 'Esplora il Machine Learning senza pre-requisiti tecnici: un corso che permette a tutti di avvicinarsi al mondo ML e AI', 'title': 'Machine Learning per tutti | Corsi Data Masters', 'source': 'https://datamasters.it/corsi/machine-learning-per-tu

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [8]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [9]:
example_messages = prompt.invoke(
    {"context": "[il contesto, composto da tutti i documenti recuperati]", "question": "[la domanda posta dall'utente]"}
).to_messages()

print(example_messages)

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: [la domanda posta dall'utente] \nContext: [il contesto, composto da tutti i documenti recuperati] \nAnswer:", additional_kwargs={}, response_metadata={})]


In [32]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: [la domanda posta dall'utente] 
Context: [il contesto, composto da tutti i documenti recuperati] 
Answer:


In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
rag_chain.invoke("mi piace l'intelligenza artificiale ma non saprei da dove iniziare a studiare, consigliami un corso")

'Ti consiglio il corso "Intelligenza Artificiale per Tutti", che è progettato specificamente per principianti senza esperienza in programmazione. Inizierai con le basi e potrai progredire verso argomenti più avanzati. Inoltre, offre un approccio pratico e immediato all\'uso dell\'AI.'

In [12]:
# recuperare i documenti selezionati per la generazione

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:
risposta = rag_chain_with_source.invoke("come potrei approfondire tematiche di Deep Learning?")

In [15]:
risposta['answer']

'Per approfondire le tematiche di Deep Learning, considera di seguire corsi specifici sul machine learning e di partecipare a programmi di formazione che offrono competenze pratiche. È utile applicare le tue conoscenze in contesti reali, come lo sviluppo di bot o progetti aziendali. Inoltre, esplora risorse che distinguono tra intelligenza artificiale, machine learning e deep learning, per avere un quadro completo.'

In [18]:
risposta['context']

[Document(metadata={'description': 'Esplora il Machine Learning senza pre-requisiti tecnici: un corso che permette a tutti di avvicinarsi al mondo ML e AI', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/machine-learning-per-tutti/', 'start_index': 7475, 'title': 'Machine Learning per tutti | Corsi Data Masters'}, page_content="e deep learning, comprendendo il ruolo dell'interazione umana.Data ScienceImparerai a classificare gli algoritmi di machine learning utilizzati per l'analisi e la modellazione dei dati.ENTRA ORAImpara dai migliori nel campo del machine learning"),
 Document(metadata={'description': 'Esplora il Machine Learning senza pre-requisiti tecnici: un corso che permette a tutti di avvicinarsi al mondo ML e AI', 'language': 'it-IT', 'source': 'https://datamasters.it/corsi/machine-learning-per-tutti/', 'start_index': 1584, 'title': 'Machine Learning per tutti | Corsi Data Masters'}, page_content='Esplorerai anche la distinzione tra intelligenza artificiale, ma